In [1]:
import requests
from IPython.display import Image


In [2]:
import requests 
from bs4 import BeautifulSoup

URL = 'https://www.atacadao.com.br/catalogo'
website = requests.get(URL)

results = BeautifulSoup(website.content, 'html.parser')

results.text

# results.splitlines()

'Atacadão –suas compras de supermercado online com preço de atacado | Atacadão\nApp Meu AtacadãoNossas lojasFolhetosCartão AtacadãoAtendimentoInstitucionalBaixe o App Meu AtacadãoBem-vindoEntre ou cadastre-se0Compre por categoriaBebidasMerceariaGrãosLimpezaHigieneÓleosBebêsPadariaBomboniereTodas as ofertasLoja - Vila Maria | Entregar no CEP:02170-901AlterarCatalogoCatalogoCatalogo0 produto encontradoOrdenar porRelevânciaFiltrar0 produto encontradoAtendimentoMeus pedidosCentral de atendimentoCartão AtacadãoFale conoscoInstitucionalQuem somosNossas lojasFolhetosTrabalhe conoscoAplicativo Meu AtacadãoImprensaBlog AtacadãoPolíticas AtacadãoPolítica de Trocas e DevoluçõesTermos de UsoPolíticas de PrivacidadePolítica de DiversidadeFormas de pagamentoBaixe o aplicativo Meu AtacadãoAcompanhe a gente nas redes sociaisRacismo é crime. Denuncie. Disque 100 ou procure a Delegacia de Polícia Civil mais próxima ou o Ministério Público.Atacadão S.A. Avenida Morvan Dias de Figueiredo, 6169, Vila Maria

In [3]:
# response = requests.get(f'https://restcountries.com/v3.1/all?fields=name,capital,continents,area,population')

# #response = requests.get('https://restcountries.com/v3.1/name/Brazil')

# response.json()

In [4]:

# response = requests.get(f'https://dog.ceo/api/breeds/image/random')

# Image(response.json()['message'])

In [5]:
# from selenium import webdriver
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By

# options = webdriver.ChromeOptions()
# options.set_capability("goog:loggingPrefs", {"performance": "ALL"})
# options.add_experimental_option("detach", True),
# options.add_argument(
#     r"--user-data-dir=C:\Users\99810903\AppData\Local\Google\Chrome\User Data\Profile 1") #abre o chrome, digita chrome://version/ e copia o "caminho do perfil"
# options.add_experimental_option("perfLoggingPrefs", {"enableNetwork": True})

# browser = webdriver.Chrome(options=options)

# delay = 5

# for page in range(1,5):

#     browser.get(f"https://www.atacadao.com.br/catalogo?productClusterIds=138&facets=productClusterIds&sort=score_desc&page={str(page)}")

#     wrapper = WebDriverWait(browser, delay).until(
#         EC.presence_of_element_located((By.XPATH,r'//*[@id="__next"]/main/section/div[2]/div[2]/div[3]/div[1]/ul/li[2]/article/div/section/h3'))
#         )

#     nav = browser.find_elements(By.TAG_NAME, 'h3')
#     preços = [text for text in browser.find_elements(By.TAG_NAME, 'p') if text.text.startswith('R$')]

#     print(f'pagina {page}')
#     for item in zip(nav, preços):
#         print(item[0].text, ' - ', item[1].text)

In [6]:
import pandas as pd
import polars as pl
from datetime import datetime

produtos = requests.get('https://www.atacadao.com.br/api/graphql?operationName=ProductsQuery&variables=%7B%22first%22%3A20%2C%22after%22%3A%220%22%2C%22sort%22%3A%22score_desc%22%2C%22term%22%3A%22%22%2C%22selectedFacets%22%3A%5B%7B%22key%22%3A%22productClusterIds%22%2C%22value%22%3A%22138%22%7D%2C%7B%22key%22%3A%22channel%22%2C%22value%22%3A%22%7B%5C%22salesChannel%5C%22%3A%5C%222%5C%22%2C%5C%22regionId%5C%22%3A%5C%22v2.58AF41F9B1F50018725A885A7CB8C910%5C%22%2C%5C%22seller%5C%22%3A%5C%22atacadaobr8%5C%22%7D%22%7D%2C%7B%22key%22%3A%22locale%22%2C%22value%22%3A%22pt-BR%22%7D%5D%7D')

quantidade = produtos.json()['data']['search']['products']['pageInfo']['totalCount']

quantidade

962

In [7]:
import pandas as pd
import polars as pl
from datetime import datetime

produtos_atacadao = requests.get('https://www.atacadao.com.br/api/graphql?operationName=ProductsQuery&variables=%7B%22first%22%3A100%2C%22after%22%3A%220%22%2C%22sort%22%3A%22score_desc%22%2C%22term%22%3A%22%22%2C%22selectedFacets%22%3A%5B%7B%22key%22%3A%22productClusterIds%22%2C%22value%22%3A%22138%22%7D%2C%7B%22key%22%3A%22region-id%22%2C%22value%22%3A%22U1cjYXRhY2FkYW9icjg%3D%22%7D%2C%7B%22key%22%3A%22channel%22%2C%22value%22%3A%22%7B%5C%22salesChannel%5C%22%3A%5C%222%5C%22%2C%5C%22regionId%5C%22%3A%5C%22v2.58AF41F9B1F50018725A885A7CB8C910%5C%22%2C%5C%22seller%5C%22%3A%5C%22atacadaobr8%5C%22%7D%22%7D%2C%7B%22key%22%3A%22locale%22%2C%22value%22%3A%22pt-BR%22%7D%5D%7D')

quantidade = produtos_atacadao.json()['data']['search']['products']['pageInfo'].get('totalCount')
#produtos_atacadao.json()
x = 0
quant_por_pagina = 100
resultados = []

while x < quantidade:
    if x + quant_por_pagina > quantidade:
        final = quantidade
    else:
        final = x + quant_por_pagina
    print(f'baixando itens {x+1} até {final}')

    produtos_var = requests.get(f'https://www.atacadao.com.br/api/graphql?operationName=ProductsQuery&variables=%7B%22first%22%3A{quant_por_pagina}%2C%22after%22%3A%22{x}%22%2C%22sort%22%3A%22score_desc%22%2C%22term%22%3A%22%22%2C%22selectedFacets%22%3A%5B%7B%22key%22%3A%22productClusterIds%22%2C%22value%22%3A%22138%22%7D%2C%7B%22key%22%3A%22channel%22%2C%22value%22%3A%22%7B%5C%22salesChannel%5C%22%3A%5C%222%5C%22%2C%5C%22regionId%5C%22%3A%5C%22v2.58AF41F9B1F50018725A885A7CB8C910%5C%22%2C%5C%22seller%5C%22%3A%5C%22atacadaobr8%5C%22%7D%22%7D%2C%7B%22key%22%3A%22locale%22%2C%22value%22%3A%22pt-BR%22%7D%5D%7D')
    
    produtos = [x['node'] for x in produtos_atacadao.json()['data']['search']['products']['edges']]
    df = pd.json_normalize(produtos, record_path= ['image'],
                       meta = ['id', 'sku', ['brand', 'name'], 'name', ['offers', 'highPrice'], ['offers', 'lowPrice']])
    df['data'] = datetime.today()
    df.columns = ['imagem', 'e', 'id', 'cod_sku_rede', 'fabricante', 'nome_prod', 'preço', 'preço_promocional', 'data']
    df = df.drop('e', axis=1)

    resultados.append(df)
    x = x + quant_por_pagina

df_consolidado = pd.concat(resultados).reset_index().drop('index', axis=1).drop_duplicates(keep='first')
df_consolidado.to_csv(f'atacadao_{datetime.today().day}-{datetime.today().month}.csv')

baixando itens 1 até 100
baixando itens 101 até 200
baixando itens 201 até 300
baixando itens 301 até 400
baixando itens 401 até 500
baixando itens 501 até 600
baixando itens 601 até 700
baixando itens 701 até 800
baixando itens 801 até 900
baixando itens 901 até 962
